## Variational Quantum Generator

In [1]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, transpile
from qiskit.tools.visualization import circuit_drawer
from qiskit.quantum_info import state_fidelity
from qiskit import BasicAer, Aer

import numpy as np
import math
from tensorflow import keras as ks
import tensorflow as tf

import matplotlib.pyplot as plt

In [2]:
class DataGenerator():
    
    def __init__(self, n_points=500):
        mu_1 = 0.20
        sigma_1 = 0.03
        mu_2 = 0.40
        sigma_2 = 0.04
        self.n_points = n_points
        self.distribution = np.append(np.random.normal(mu_1, sigma_1, int(self.n_points / 2)), np.random.normal(mu_2, sigma_2, int(self.n_points / 2)))
    

In [3]:
class ClipConstraint(ks.constraints.Constraint):
    # set clip value when initialized
    def __init__(self, clip_value):
        self.clip_value = clip_value

    # clip model weights to hypercube
    def __call__(self, weights):
        return ks.backend.clip(weights, -self.clip_value, self.clip_value)

    # get the config
    def get_config(self):
        return {'clip_value': self.clip_value}
    
# implementation of wasserstein loss
def wasserstein_loss(y_true, y_pred):
    return ks.backend.mean(y_true * y_pred)

In [22]:
class VariationalQuantumGAN():

    def __init__(self, n_epochs=50):
        super().__init__()
        # Initialize parameters
        self.backend = Aer.get_backend('statevector_simulator')
        self.backend_sim = Aer.get_backend('qasm_simulator')
        self.latent_space_size = 2
        self.variational_circuit_size = 2
        
        # Training parameters
        self.n_epochs = n_epochs
        
        # Parameters initialized as in the paper.
        self.qg_thetas = [2.3, 2.3, 1.0, 1.0, 1.5, 0.2]
        # Build circuit with class parameters TODO: Use this to improve efficiency (if needed)
        self.qc = self.build_circuit()
        
        self.decoding_model = None
        self.build_decoding_model()
        
        self.data_generator = DataGenerator()
        self.target_dist = self.data_generator.distribution
        
        self.discriminator = self.build_discriminator()
        
        # Build gan stack for fitting the generator
        self.gan_stack = ks.Sequential()
        self.gan_stack.add(self.decoding_model)
        self.gan_stack.add(self.discriminator)
        # TODO: Check learning rate
        opt = ks.optimizers.RMSprop(lr=0.00005)
        self.gan_stack.compile(loss=wasserstein_loss, optimizer=opt)
        
    def build_circuit(self, measurement=True):
        qc = QuantumCircuit(self.variational_circuit_size, self.variational_circuit_size)
    
        # Quantum encoding.
        # z ~ U[-1, 1]
        z = np.random.uniform(-1, 1, self.latent_space_size)

        for i in range(self.latent_space_size):
            qc.rx(1/math.sin(z[i]), i + (self.variational_circuit_size - self.latent_space_size))
            qc.rz(1/math.cos(z[i]), i + (self.variational_circuit_size - self.latent_space_size))

        qc.barrier()

        # Variational Circuit
        # TODO: Scale dynamically with circuit size?
        qc.ry(self.qg_thetas[0], 0)
        qc.ry(self.qg_thetas[1], 1)
        qc.rxx(self.qg_thetas[2], 0, 1)
        qc.barrier()
        
        qc.ry(self.qg_thetas[3], 0)
        qc.ry(self.qg_thetas[4], 1)
        qc.rxx(self.qg_thetas[5], 0, 1)
        qc.barrier()

        # Measurement decoding
        if measurement:
            qc.measure(0, 0)
            qc.measure(1, 1)
        
        return qc
    
    # Build circuit with params as arguments, used for parameter shift gradient calculation
    def build_circuit_with_params(self, params, measurement=True):
        qc = QuantumCircuit(self.variational_circuit_size, self.variational_circuit_size)
    
        # Quantum encoding.
        # z ~ U[-1, 1]
        z = np.random.uniform(-1, 1, self.latent_space_size)

        for i in range(self.latent_space_size):
            qc.rx(1/math.sin(z[i]), i + (self.variational_circuit_size - self.latent_space_size))
            qc.rz(1/math.cos(z[i]), i + (self.variational_circuit_size - self.latent_space_size))

        qc.barrier()

        # Variational Circuit
        # TODO: Scale dynamically with circuit size?
        qc.ry(params[0], 0)
        qc.ry(params[1], 1)
        qc.rxx(params[2], 0, 1)
        qc.barrier()
        
        qc.ry(params[3], 0)
        qc.ry(params[4], 1)
        qc.rxx(params[5], 0, 1)
        qc.barrier()

        # Measurement decoding
        if measurement:
            qc.measure(0, 0)
            qc.measure(1, 1)
        
        return qc
    
    def build_discriminator(self):
        # define the constraint
        const = ClipConstraint(0.01)
        
        model = ks.Sequential()
        model.add(ks.layers.Dense(50, activation='elu', input_shape=(1,), kernel_constraint=const))
#         model.add(ks.layers.Dense(100, activation='relu', kernel_constraint=const))
        model.add(ks.layers.Dense(50, activation='elu', kernel_constraint=const))
        model.add(ks.layers.Dense(1, activation='linear', kernel_constraint=const))
        
        # TODO: Investigate loss
        opt = ks.optimizers.RMSprop(lr=0.005)
        model.compile(loss=wasserstein_loss, optimizer=opt)
        return model
    
    def build_decoding_model(self):
        x = ks.layers.Input(shape=(self.variational_circuit_size,), name="input")
        decoding_layer = ks.layers.Dense(1, activation="sigmoid", name="decoding_layer")
        y = decoding_layer(x)

        self.decoding_model = ks.Model(inputs=x, outputs=y)
        self.decoding_model.summary()
        
    # Generate measurement with the class 
    def generate_measurement(self, qc=None, statevector=False):
        # Somehow statevector is slower: TODO figure out. 25 seconds sv vs 15 seconds 100 shot sims.
        if not statevector:
            # TODO: change to use a class qc
            if qc is None:
                qc = self.build_circuit()
            sim_shots = 100

            job = self.backend_sim.run(qc, shots=sim_shots)
            result_sim = job.result()

            counts = result_sim.get_counts(qc)
    #         print(counts)

            classical_latent_space = np.zeros(self.variational_circuit_size)
            for count in counts.keys():
                v_n = 0
                for bit in count:
                    if bit == '1':
                        classical_latent_space[v_n] += counts[count]
                    v_n += 1

            classical_latent_space = np.stack([classical_latent_space/sim_shots])

            return classical_latent_space
        
        else:
            if qc is None:
                qc_no_measure = self.build_circuit(measurement=False)
                qc_no_measure.save_statevector()
            else:
                qc_no_measure = qc
            

            # Transpile for simulator
            simulator = Aer.get_backend('aer_simulator')
            circ = transpile(qc_no_measure, simulator)

            # Run and get statevector
            result = simulator.run(circ).result()
            statevector = result.get_statevector(circ)

            collapsed_sv = np.abs(np.stack(statevector)) ** 2

            classical_latent_space = np.zeros(self.variational_circuit_size)
            for i in range(len(collapsed_sv)):
                # Counts are fixed. CHANGE 02b TO NEW LATENT SPACE SIZE IF CHANGED
                current_count = "{0:02b}".format(i)
                for bit in range(len(current_count)):
                    if current_count[bit] == "1":
                        classical_latent_space[bit] += collapsed_sv[i]

    #             000 , 001, 010, 011, 100, 101, 110, 111

    #         print(classical_latent_space)
    #         print(collapsed_sv)

            classical_latent_space = classical_latent_space.reshape(1, self.variational_circuit_size)

            return classical_latent_space
        
    def generate_prediction(self, qc=None):
        circuit_measurement = self.generate_measurement(qc=qc)
        return self.decoding_model.predict(circuit_measurement)
    
    def generate_fake_samples(self, n):
        print("Generating fake samples for training")
        predictions = []
#         self.printProgressBar(iteration=0, total=10)
        j = 0
        for i in range(n):
            predictions.append(self.generate_prediction())
#             if i%(int(n/10)) == 0:
#                 j += 1
#                 self.printProgressBar(iteration=j, total=10)
            
        predictions = np.stack(predictions)
        predictions = predictions.reshape((n, 1))
        
        return predictions, np.ones((n, 1))
    
    def generate_fake_samples(self, n, qc=None):
        print("Generating fake samples for training")
        predictions = []
#         self.printProgressBar(iteration=0, total=10)
        j = 0
        for i in range(n):
            predictions.append(self.generate_prediction(qc=qc))
#             if i%(int(n/10)) == 0:
#                 j += 1
#                 self.printProgressBar(iteration=j, total=10)
            
        predictions = np.stack(predictions)
        predictions = predictions.reshape((n, 1))
        
        return predictions, np.ones((n, 1))
    
    def train(self):
        self.discriminator.summary()
        
        # TODO: Resample real distribution continuously instead of using fixed amount of points
        X_real = np.stack(self.target_dist).reshape(len(self.target_dist), 1)
        y_real = -np.ones((len(self.target_dist), 1))
        
        n_critic = 10
        gen_batch_size = 32
        disc_batch_size = gen_batch_size * n_critic * 2
        
        for i in range(self.n_epochs):
            X_fake_sampled, y_fake_sampled = self.generate_fake_samples(int(disc_batch_size / 2))
            # Train discriminator
            # TODO: Perform basic improvements on discriminator
            # TODO: Create wasserstein critic
            
            # Select read and fake data_points, we assume N_fake == N_real (Amount fake datapoints is equal to amount of real datapoints)
            idx = np.random.choice(np.arange(len(X_real)), int(disc_batch_size / 2))
            
            # Train on real
            X_real_sampled, y_real_sampled = np.take(X_real, idx), np.take(y_real, idx)
            
#             preds = self.discriminator.predict_on_batch
            self.discriminator.train_on_batch(X_real_sampled, y_real_sampled)
            
            # Train on fake
            self.discriminator.train_on_batch(X_fake_sampled, y_fake_sampled)
            
            print("Iteration", i)#, "Real accuracy", acc_real, "Fake accuracy", acc_fake)
            
            # Train generator stack
            # TODO: Figure out gradients exactly
            
            # Calculate Generator Cost
            disc_pred = 0
            for j in range(len(X_fake_sampled)):
                disc_pred += self.discriminator.predict(X_fake_sampled[j])/disc_batch_size
                
            print("Gen cost:", disc_pred)
            self.generate_critic_belief(plot=True, iteration=i)
            
            self.discriminator.trainable = False
            # Update Quantum and Classical part of generator with parameter shift
            for j in range(gen_batch_size):
                non_shift_qc = self.build_circuit_with_params(self.qg_thetas)
                non_shift_measurement = self.generate_measurement(qc=non_shift_qc)
#                 print(non_shift_measurement)
                non_shift_sample = self.decoding_model.predict(non_shift_measurement)
                non_shift_cost = self.discriminator.predict([non_shift_sample])
                
                # TODO: Check .astype float for tf retracing inefficiency
                self.gan_stack.train_on_batch(np.stack(non_shift_measurement), np.stack([1]).astype(float))
                
                for k in range(len(self.qg_thetas)):
                    neg_shift_params = self.qg_thetas.copy()
                    neg_shift_params[k] -= math.pi/2
                    neg_shift_qc = self.build_circuit_with_params(neg_shift_params)
                    
                    neg_shift_sample = self.generate_prediction(qc=neg_shift_qc)
                    neg_shift_cost = self.discriminator.predict([neg_shift_sample])
                    
                    pos_shift_params = self.qg_thetas.copy()
                    pos_shift_params[k] -= math.pi/2
                    pos_shift_qc = self.build_circuit_with_params(pos_shift_params)
                    
                    pos_shift_sample = self.generate_prediction(qc=pos_shift_qc)
                    pos_shift_cost = self.discriminator.predict([pos_shift_sample])
                    
                    gradient = pos_shift_cost - neg_shift_cost
                    
                    self.qg_thetas[k] += gradient * non_shift_cost
            
            
            self.discriminator.trainable = True
            
            if i % 5 == 0:
                self.compare_model_to_real()
                
    def generate_critic_belief(self, plot=False, iteration=-1):
        values = np.arange(0, 1, 0.01)
        beliefs = []
        for i in range(len(values)):
            beliefs.append(self.discriminator.predict([values[i]]))
        
        beliefs = np.reshape(beliefs, (100))
        
        if plot:
            plt.plot(np.arange(0, 1, 0.01), beliefs, label="belief")
            plt.title("Belief at iteration " + str(iteration))
#             plt.ylim(-1, 1)
            plt.show()
            
        return beliefs
            
    def compare_model_to_real(self):
        bins = np.linspace(0, 1, 100)
        
        plt.hist(self.target_dist, bins, density=True, label="True")
        predictions = []
        self.printProgressBar(iteration=0, total=10)
        j = 0
        for i in range(self.data_generator.n_points):
            predictions.append(self.generate_prediction())
            if i%(self.data_generator.n_points/10) == 0:
                j += 1
                self.printProgressBar(iteration=j, total=10)
#                 print(str(i/self.data_generator.n_points*100) + "%")
                
        predictions = np.stack(predictions)
        predictions = predictions.reshape((self.data_generator.n_points,))

        plt.hist(predictions, bins, density=True, label="False")
        
        plt.plot(np.arange(0, 1, 0.01), self.generate_critic_belief(), label="Critic belief")
        
        plt.legend()
        plt.show()
        
    def printProgressBar (self, iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█', printEnd = "\r"):
        percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
        filledLength = int(length * iteration // total)
        bar = fill * filledLength + '-' * (length - filledLength)
        print(f'\r{prefix} |{bar}| {percent}% {suffix}', end = printEnd)
        # Print New Line on Complete
        if iteration == total: 
            print()
            
    
vqgan = VariationalQuantumGAN()
# vqgan.train()
# vqgan.generate_prediction()

Model: "model_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 2)]               0         
_________________________________________________________________
decoding_layer (Dense)       (None, 1)                 3         
Total params: 3
Trainable params: 3
Non-trainable params: 0
_________________________________________________________________


In [29]:
import math
import numpy as np

latent_space_size = 2
variational_circuit_size = 2

# Training parameters
n_epochs = 5

# Parameters initialized as in the paper.
qg_thetas = [2.3, 2.3, 1.0, 1.0, 1.5, 0.2]

qc = QuantumCircuit(variational_circuit_size, variational_circuit_size)

# Quantum encoding.
# z ~ U[-1, 1]
z = np.random.uniform(-1, 1, latent_space_size)

for i in range(latent_space_size):
    qc.rx(1/math.sin(z[i]), i + (variational_circuit_size - latent_space_size))
    qc.rz(1/math.cos(z[i]), i + (variational_circuit_size - latent_space_size))

qc.barrier()

# Variational Circuit
# TODO: Scale dynamically with circuit size?
qc.ry(qg_thetas[0], 0)
qc.ry(qg_thetas[1], 1)
qc.rxx(qg_thetas[2], 0, 1)
qc.barrier()

qc.ry(qg_thetas[3], 0)
qc.ry(qg_thetas[4], 1)
qc.rxx(qg_thetas[5], 0, 1)
qc.barrier()

# Measurement decoding
qc.measure(0, 0)
qc.measure(1, 1)

qc.draw()

┌────────────┐┌───────────┐  ░ ┌─────────┐┌─────────┐ ░  ┌───────┐ »
q_0: ─┤ Rx(-1.508) ├┤ Rz(1.336) ├──░─┤ Ry(2.3) ├┤0        ├─░──┤ Ry(1) ├─»
     ┌┴────────────┤├───────────┴┐ ░ ├─────────┤│  Rxx(1) │ ░ ┌┴───────┴┐»
q_1: ┤ Rx(-7.0049) ├┤ Rz(1.0103) ├─░─┤ Ry(2.3) ├┤1        ├─░─┤ Ry(1.5) ├»
     └─────────────┘└────────────┘ ░ └─────────┘└─────────┘ ░ └─────────┘»
c: 2/════════════════════════════════════════════════════════════════════»
                                                                         »
«     ┌───────────┐ ░ ┌─┐   
«q_0: ┤0          ├─░─┤M├───
«     │  Rxx(0.2) │ ░ └╥┘┌─┐
«q_1: ┤1          ├─░──╫─┤M├
«     └───────────┘ ░  ║ └╥┘
«c: 2/═════════════════╩══╩═
«                      0  1